<a href="https://colab.research.google.com/github/GUNAPILLCO/neural_profit/blob/main/2_obtencion_preparacion_exploracion_datos/2_4_0_alpha_factors_hipotesis%20-%20copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.4. Generación de alpha factors por hipotesis

## 0. Clonado de repositorio e importación de librerías

In [1]:
#Clonamos el repo
#LINK DE REPOSITORIO: https://github.com/GUNAPILLCO/neural_profit

!git clone https://github.com/GUNAPILLCO/neural_profit.git

Cloning into 'neural_profit'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 144 (delta 32), reused 24 (delta 14), pack-reused 89 (from 1)
Receiving objects: 100% (144/144), 108.42 MiB | 10.26 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Updating files: 100% (39/39), done.


In [2]:
import sys

#Instalación de librería pandas_market_calendars
#!{sys.executable} -m pip install -q pandas_market_calendars
#print("Librería instalada: pandas_market_calendars")

# Utilidades generales
from datetime import datetime, timedelta
import os
import glob
import requests
import warnings
warnings.filterwarnings('ignore')

# Manejo y procesamiento de datos
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
# Calendario de mercados
#import pandas_market_calendars as mcal

## 2. Carga del dataset mnq_intraday_data

In [3]:
def load_df():
    """
    Función para cargar un archivo Parquet desde el repositorio clonado
    """
    # Definir la URL del archivo Parquet en GitHub
    df_path = '/content/neural_profit/2_obtencion_preparacion_exploracion_datos/mnq_intraday_data.parquet'

    # Leer el archivo Parquet y cargarlo en un DataFrame
    df = pd.read_parquet(df_path)

    # Asegurar que el índice esté en formato datetime
    df.index = pd.to_datetime(df.index)

    # Crear una nueva columna 'date' con la fecha extraída del índice
    df['date'] = df.index.date

    # Reordenar columnas: 'date', 'time_str', y luego el resto
    cols = ['date'] + [col for col in df.columns if col not in ['date']]

    df = df[cols]

    return df

In [4]:
mnq_intraday = load_df()

## 3. Agregar Alpha Factors

### 3.0. Funciones

**Función para evaluar alpha factor**

In [5]:
def evaluar_alpha_factor(df: pd.DataFrame,
                         factor_generador_fn,
                         target_minutes=[5, 15, 30],
                         factor_prefix="factor") -> pd.DataFrame:
    """
    Evalúa el poder predictivo (IC promedio por día) de un alpha factor generado.

    Parámetros:
    - df: DataFrame con columnas 'date', 'close', 'volume'
    - factor_generador_fn: función que recibe el df y target_minutes y agrega las columnas del alpha factor
    - target_minutes: ventanas de tiempo para calcular retornos futuros
    - factor_prefix: prefijo común de las columnas del factor (ej. 'momentum_z_', 'reversal_score_', etc.)

    Retorna:
    - DataFrame con el IC promedio para cada ventana de tiempo
    """
    df = df.copy()

    # 1. Calcular retornos futuros (target)
    for N in target_minutes:
        df[f'target_return_{N}'] = df.groupby('date')['close'].transform(
            lambda x: np.log(x.shift(-N)) - np.log(x)
        )

    # 2. Aplicar la función que genera el alpha factor
    df = factor_generador_fn(df, target_minutes)

    # 3. Calcular IC por día para cada ventana
    ic_results = {}
    for N in target_minutes:
        factor_col = f'{factor_prefix}{N}'
        target_col = f'target_return_{N}'
        ic_series = df.dropna(subset=[factor_col, target_col]).groupby('date').apply(
            lambda x: x[factor_col].corr(x[target_col])
        )
        ic_results[f'IC_{N}'] = ic_series.mean()

    return pd.DataFrame.from_dict(ic_results, orient='index', columns=['Mean IC'])


**Función para observar el comportamiento del alpha factor durante el día**

### 3.1. Hipotesis 1

In [6]:
#Hipotesis 1
def generar_momentum_z(df, ventanas):
    for N in ventanas:
        col_raw = f'momentum_{N}'
        col_z = f'momentum_z_{N}'
        df[col_raw] = df.groupby('date')['close'].transform(lambda x: np.log(x) - np.log(x.shift(N)))
        df[col_z] = df.groupby('date')[col_raw].transform(lambda x: (x - x.mean()) / x.std())
    return df

In [13]:
evaluar_alpha_factor(mnq_intraday, generar_momentum_z, factor_prefix="momentum_z_")


,Mean IC
IC_5,-0.025171
IC_15,-0.049925
IC_30,-0.099064


In [7]:
#Hipotesis 2
def generar_reversal_score(df, ventanas):
    for N in ventanas:
        sma = f'sma_{N}'
        std = f'std_{N}'
        z = f'zscore_{N}'
        rev = f'reversal_score_{N}'
        df[sma] = df.groupby('date')['close'].transform(lambda x: x.rolling(N, 1).mean())
        df[std] = df.groupby('date')['close'].transform(lambda x: x.rolling(N, 1).std())
        df[z] = (df['close'] - df[sma]) / df[std]
        df[rev] = df.groupby('date')[z].transform(lambda x: (x - x.mean()) / x.std())
    return df

In [8]:
#Hipotesis 3
def generar_momentum_vol_z(df, ventanas):
    for N in ventanas:
        mom = f'momentum_{N}'
        vavg = f'vol_avg_{N}'
        vr = f'vol_ratio_{N}'
        factor = f'mom_vol_{N}'
        z = f'mom_vol_z_{N}'
        df[mom] = df.groupby('date')['close'].transform(lambda x: np.log(x) - np.log(x.shift(N)))
        df[vavg] = df.groupby('date')['volume'].transform(lambda x: x.rolling(N, 1).mean())
        df[vr] = df['volume'] / df[vavg]
        df[factor] = df[mom] * df[vr]
        df[z] = df.groupby('date')[factor].transform(lambda x: (x - x.mean()) / x.std())
    return df

,Mean IC
IC_5,-0.025171
IC_15,-0.049925
IC_30,-0.099064


In [10]:
evaluar_alpha_factor(mnq_intraday, generar_reversal_score, factor_prefix="reversal_score_")


,Mean IC
IC_5,-0.018276
IC_15,-0.039496
IC_30,-0.069822


In [11]:
evaluar_alpha_factor(mnq_intraday, generar_momentum_vol_z, factor_prefix="mom_vol_z_")

,Mean IC
IC_5,-0.024858
IC_15,-0.049027
IC_30,-0.086046


### 3.1. De hipótesis 1: Momentum factor

In [ ]:
def add_momentum_factor(df: pd.DataFrame, window: int = 30, price_col: str = 'close') -> pd.DataFrame:

    ## De la hipotesis 1  momentum_z_30

    """
    Agrega un alpha factor de momentum normalizado (z-score), basado en log-retornos pasados
    con una ventana deslizante por jornada. La columna resultante se llama 'momentum_factor'.

    Este factor representa la hipótesis de continuación del momentum intradía.

    Parámetros:
    - df: DataFrame con columnas 'date' y la columna de precios.
    - window: Minutos hacia atrás para calcular el log-retorno (default = 30).
    - price_col: Columna del precio (default = 'close').

    Retorna:
    - DataFrame con una nueva columna: 'momentum_factor'.
    """

    df = df.copy()

    df['momentum'] = df.groupby('date')[price_col].transform(lambda x: np.log(x) - np.log(x.shift(window)))

    df['momentum_factor'] = df.groupby('date')['momentum'].transform(lambda x: (x - x.mean()) / x.std())

    df = df.drop(columns=['momentum'])

    return df

In [ ]:
mnq_intraday = add_momentum_factor(mnq_intraday, window=30)

### 3.2. De hipótesis 2: Mean Reversion Factor

In [ ]:
def add_mean_reversion_factor(df: pd.DataFrame, window: int = 5, price_col: str = 'close') -> pd.DataFrame:
    ## De la hipotesis 2  zcore_5

    """
    Agrega al DataFrame una única columna con el alpha factor de reversión a la media
    basado en z-score, calculado sobre una media móvil simple por jornada.

    La columna resultante se llama: 'mean_reversion_factor'.

    Parámetros:
    - df: DataFrame con columnas 'date' y 'close'.
    - window: Ventana de minutos para la media móvil (default=5).
    - price_col: Nombre de la columna de precios (default='close').

    Retorna:
    - DataFrame con una nueva columna: 'mean_reversion_factor'.
    """

    df = df.copy()

    # Función auxiliar para calcular el z-score rolling por día
    def zscore_rolling(x):
        mean = x.rolling(window=window).mean()
        std = x.rolling(window=window).std()
        return (x - mean) / std

    # Aplicar por jornada
    df['mean_reversion_factor'] = df.groupby('date')[price_col].transform(zscore_rolling)

    return df


In [ ]:

mnq_intraday = add_mean_reversion_factor(mnq_intraday, window=5)


### 3.3. De hipótesis 3: Momentum volumen factor


In [ ]:
def add_mom_vol_factor(df: pd.DataFrame, window: int = 30,
                       price_col: str = 'close', volume_col: str = 'volume',
                       date_col: str = 'date') -> pd.DataFrame:
    """
    Agrega un alpha factor basado en momentum x volumen relativo, normalizado por jornada.
    Diseñado para datasets intradía con estructura diaria independiente.

    Parámetros:
    - df: DataFrame original con columnas de precios, volumen y fecha.
    - window: Ventana (en minutos) para cálculo del momentum y volumen promedio.
    - price_col: Columna del precio (default = 'close').
    - volume_col: Columna del volumen (default = 'volume').
    - date_col: Columna que indica la jornada (default = 'date').

    Retorna:
    - DataFrame con columna adicional: 'mom_vol_z_{window}'
    """
    df = df.copy()

    momentum_col = f'momentum_{window}'
    vol_avg_col = f'vol_avg_{window}'
    vol_ratio_col = f'vol_ratio_{window}'
    factor_col = f'mom_vol_{window}'
    z_col = f'mom_vol_z_{window}'

    # Calcular momentum por día: log-retorno contra valor N minutos atrás
    df[momentum_col] = df.groupby(date_col)[price_col].transform(
        lambda x: np.log(x) - np.log(x.shift(window))
    )

    # Volumen promedio (rolling por día)
    df[vol_avg_col] = df.groupby(date_col)[volume_col].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

    # Ratio volumen actual vs promedio
    df[vol_ratio_col] = df[volume_col] / df[vol_avg_col]

    # Alpha factor (con reversión)
    df[factor_col] = -1 * df[momentum_col] * df[vol_ratio_col]

    # Normalización por día (z-score)
    df[z_col] = df.groupby(date_col)[factor_col].transform(
        lambda x: (x - x.mean()) / x.std()
    )

    return df

In [ ]:
mnq_intraday = add_mom_vol_factor(mnq_intraday, window=30)
